## Preparations

We will use `mlr` to train the model. Additionally, we use parallelization to speed up the evaluation and tuning:

In [ ]:
library(mlr)

library(parallelMap)

# Let one core remain to not completely slow down the system:
parallelStartSocket(parallel::detectCores() / 2)

## Data Import and Splitting

We use the prepared dataset `insurance_final.csv`:

In [ ]:
# With base R:
insurance = read.csv("data/insurance_final.csv")

# Remove rows with missings:
insurance = na.omit(insurance)

Alternatives to deleting data would be to impute the missing values with i.e. the mean or median. More advanced techniques here could be the EM algorithm.

In order to finally evaluate the model on the new data from 2018 we create two index vectors which are used for training and a final holdout prediction:

In [ ]:
index_train = insurance$year < 2018
index_eval  = insurance$year == 2018

Finally, we delete all features which does not contain information or are not allowed to use for modeling:

In [ ]:
insurance = insurance[, -which(colnames(insurance) %in% c("id", "id2", "sex", "year"))]

## Define the Task

To find a good regression model for `charges` we use all available features, with exception of `id`, `id2`, `sex`, and `year`, from the insurance dataset:

In [ ]:
task = makeRegrTask(id = "insurance_data", target = "charges", data = insurance[index_train, ])

## Define Learner

In [ ]:
learner_ranger = makeLearner("regr.ranger")
learner_gbm = makeLearner("regr.gbm")

## Define Resampling Strategy

In [ ]:
# Recommended but expensive:
rdesc = makeResampleDesc(method = "RepCV", folds = 10, reps = 3)

# We are using a 5-fold CV which is faster but not as accurate:
rdesc = makeResampleDesc(method = "CV", iters = 5)

# Fix instance for a later comparison:
resample_instance = makeResampleInstance(desc = rdesc, task = task)

## Define Measures

In [ ]:
used_measures = list(mae, mse, rsq)

## Define Hyperparameter Space

In [ ]:
par_set_gbm = makeParamSet(
  makeDiscreteParam("distribution", values = c("gaussian", "laplace", "tdist"))
  , makeIntegerParam("n.trees", lower = 100, upper = 3000)
  , makeNumericParam("shrinkage", lower = 0.0001, upper = 0.25)
  , makeNumericParam("bag.fraction", lower = 0.3, upper = 0.6)
  , makeIntegerParam("interaction.depth", lower = 1, upper = 10)
)

## Define Tuning Strategy

In [ ]:
# tune_ctrl = makeTuneControlRandom(maxit = 200L)
tune_ctrl = makeTuneControlRandom(maxit = 20L)

For the inner evaluation of a drawn parameter set we use 3-fold cross validation:

In [ ]:
learner_resample_gbm = makeTuneWrapper(learner = learner_gbm, resampling = cv3, measures = mse, 
  par.set = par_set_gbm, control = tune_ctrl, show.info = TRUE)

## Conduct the Benchmark/Tuning

In [ ]:
bm_res = benchmark(learners = list("regr.featureless", "regr.lm", "regr.rpart", "regr.ranger"), task = task, 
  measures = used_measures, resamplings = resample_instance)

In [ ]:
nested_tune_res_gbm = resample(learner = learner_resample_gbm, task = task, resampling = resample_instance, extract = getTuneResult, 
  show.info = FALSE, measures = used_measures, keep.pred = FALSE)

## Stop Parallelization

In [ ]:
parallelStop()

## Results

In [ ]:
bm_res
nested_tune_res_gbm

## Final Benchmark

In [ ]:
final_task = makeRegrTask(id = "insurance_data", target = "charges", data = insurance)

best_learner = lapply(X = nested_tune_res_gbm$extract, FUN = function (res) {
  learner = makeLearner(id = paste0("y_", round(res$y)), "regr.gbm")
  learner = setHyperPars(learner = learner, par.vals = res$x)
  return(learner)
})
final_holdout = makeFixedHoldoutInstance(train.inds = which(index_train), test.inds = which(index_eval), size = nrow(insurance))
final_bm = benchmark(learners = best_learner, task = final_task, measures = used_measures, resamplings = final_holdout)
final_bm

## Visualize fit on 2018

We will use `ggplot2` for visualizations:

In [ ]:
library(ggplot2)

We want to visualize the best model w.r.t. the lowers mae:

In [ ]:
index_best_learner = which.max(getBMRAggrPerformances(final_bm, as.df = TRUE)$mae.test.mean)

Finally, we fit the model on the data without 2018 and create the plot for that year:

In [ ]:
par_set = nested_tune_res_gbm$extract[[index_best_learner]]$x

task = makeRegrTask(id = "insurance_data", target = "charges", data = insurance)
learner = makeLearner("regr.gbm")
learner = setHyperPars(learner, par.vals = par_set)

final_model = train(learner = learner, task = task, subset = index_train)

pred_2018 = predict(final_model, newdata = insurance[index_eval, ])

plot_data = pred_2018$data
plot_data$Accuracy = cut(x = abs(plot_data$truth - plot_data$response), breaks = c(0, 500, 5000, Inf), labels = c("good", "ok", "critical"))

ggplot() +
  geom_point(data = plot_data, mapping = aes(x = response, y = truth, color = Accuracy), alpha = 0.5) + 
  scale_color_brewer(palette = "PuBu") +
  xlab("Predicted Premium") + ylab("Real Claims")

## Build Final Model

The last step is to build a model which uses all of the available data. This model should be used for further prediction tasks:

In [ ]:
# Build final model:
task = makeRegrTask(id = "insurance_data", target = "charges", data = insurance)
learner = makeLearner("regr.gbm")
learner = setHyperPars(learner, par.vals = nested_tune_res_gbm$extract[[index_best_learner]]$x)

final_model = train(learner = learner, task = task)